---
Why (and What) is Python?
---

### Why Python?

You might wonder why caring about accelerating your (already working) Python code is important. Let's have a look at a really simple example of PI approximation:

$$\huge{\pi = \arctan\left(1\right)}$$

$$\huge{\pi \approx 4 \times \left(\frac{1}{1} - \frac{1}{3} + \frac{1}{5} - \ldots\right)}$$


We'll use a naive implementation, both in C and Python for comparison.

Here is the C implementation, found in approx_pi.c:

In [ ]:
import os
os.chdir("%s/cq-formation-advanced-python"%os.getenv("HOME"))

In [ ]:
%%writefile approx_pi.c
#include <math.h>
#include <time.h>
#include <stdio.h>
#include <stdlib.h>

double approx_pi(int);

int main(int argc, char *argv[])
{
    clock_t t1 = clock();
    int n = atoi(argv[1]);
    double pi = approx_pi(n);
    clock_t t2 = clock();
    printf("PI is approximately %.16f, Error is %.16f\n",
	   pi, fabs(pi - M_PI));
    printf("Time = %.16f sec\n", (double)(t2 - t1)/CLOCKS_PER_SEC);
    return 0;
}

double approx_pi(int intervals)
{
     double pi = 0.0;
     int i;
     for (i = 0; i < intervals; i++) {
         // Note: using "float" instead of "double" is a little faster here
         pi += (4 - ((i % 2) * 8)) / (double)(2 * i + 1);
     }
     return pi;
}


And here is the equivalent Python implementation, found in approx_pi.py:

In [ ]:
%%writefile approx_pi.py
import sys
import math
import time

def approx_pi(intervals):
    pi = 0.0
    for i in range(intervals):
        pi += (4 - 8 * (i % 2)) / (float)(2 * i + 1)
    return pi

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print >> sys.stderr, "usage: {0} <intervals>".format(sys.argv[0])
        sys.exit(1)

    t1 = time.clock()
    pi = approx_pi(int(sys.argv[1]))
    t2 = time.clock()
    print("PI is approximately %.16f, Error is %.16f"%(pi, abs(pi - math.pi)))
    print("Time = %.16f sec\n"%(t2 - t1))


In [ ]:
def approx_pi(intervals):
    pi = 0.0
    for i in range(intervals):
        pi += (4 - 8 * (i % 2)) / (float)(2 * i + 1)
    return pi

Compiling and running the resulting binary would result in something like this:

In [ ]:
!make approx_pi && ./approx_pi 100000000

Be aware that your measurements can be significantly different on different computers. Now for its Python counterpart:

In [ ]:
%run approx_pi.py 100000000

You will usually notice that the C version is 80 to 100 times faster than the Python code. You can imagine that when running long simulation, this can make a huge difference.

We can also use %time and %timeit to measure performance: %timeit returns a best of 3,
meaning it repeats 3 times calling the function in a loop and returns the best time.
We will see how to modify this behaviour a bit later

In [ ]:
%time approx_pi(100000)

In [ ]:
%timeit approx_pi(100000)

### What is Python?

Let's take a step back and answer that question: What is Python?

It is self-described as "Python is a programing language that lets you work quickly and integrate systems more effectively", which is quite accurate. From a development standpoint, designing, writing and deploying Python code is usually faster than the equivalent C code. As we have seen, the only problem is that you have to make some kind of effort to speed it up.

The first thing to know is that when talking about Python, most people are really talking about [CPython](https://python.org). CPython is the reference implementation of the language but there are others. From the words of its authors, CPython is not always the fastest implementation. Let's try using one of them, [PyPy](http://pypy.org/), to see how they compare.

Using the same Python code above, we can run it using the PyPy interpreter:

In [ ]:
! /software6/apps/pypy/4.0.1/bin/pypy-c approx_pi.py 100000000

That was fast. We are still about twice slower than C, but it's not bad, taking into account we did not have to change a single line of code. It's important to note that PyPy has its [limits](http://pypy.org/compat.html):

1. It is only Python 2.7.12/3.3.5 compliant, not 3.4/3.5 (yet).
2. It supports most of the standard library modules.
3. It doesn't support, out of the box, C Python extensions.
4. Since Numpy is a C extension, they maintain a [PyPy-compatible fork](https://bitbucket.org/pypy/numpy.git).

Let's return to a part the above statement: "integrate systems more effectively".

Python should really be seen as the glue keeping together multiple components (libraries). And since pure Python is usually pretty slow, compared to C, that means you want to integrate components written in C, and spend most of your computation time inside those libraries, not in your Python code.

### Numpy

Numpy is a big part of speeding up Python scientific software. Numpy is not the subject of study for this workshop but it is worth mentioning how simple loops can be replaced with Numpy arrays. For example the following equation:

$$ {\Large \sum_{i=0}^{N-1} i/10} $$

Could be represented as the following Python code:

In [ ]:
N = 100

s = 0.
for i in range(N):
    s += i/10.

s

And could be replaced by this single, very efficient, statement, after first importing numpy:

In [ ]:
import numpy

In [ ]:
s = numpy.sum(numpy.arange(N)/10.)
s

Or, of course:

In [ ]:
N*(N-1)/20.0
s

> ## Using Numpy {.challenge}
> 
> Use Numpy's _arange_ function to speed up the following code, and check with %timeit:
>

In [ ]:
def approx_pi(intervals):
     pi = 0.0
     for i in range(intervals):
         pi += (4 - 8 * (i % 2)) / (float)(2 * i + 1)
     return pi

> A solution can be found in the solutions/approx_pi_numpy.py file.

In [ ]:
%load solutions/approx_pi_numpy.py

Running the solution for the previous challenge would yield this result:

In [ ]:
%run solutions/approx_pi_numpy.py 100000000

While not as good as with the PyPy interpreter, it's still quite an improvement compared to our original version. The advantage of using Numpy is that you maintain full compatibility with the official Python release. This is something you want to take into account when deciding whether you should use the PyPy interpreter. It all depends on the libraries you are using and the level of compatibility vs. performance you want to achieve.